In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import importlib

In [2]:
import sys
import os

# Get the absolute path to the 'src' directory
sys.path.append(os.path.abspath("../.."))
print(os.path.abspath("../.."))
project_root = os.path.abspath("../..")
print(sys.path)
print(project_root)

/home/julia/school/bakalarka/hospital_sim
['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/julia/school/bakalarka/simVenv/lib/python3.12/site-packages', '/home/julia/school/bakalarka/hospital_sim']
/home/julia/school/bakalarka/hospital_sim


In [ ]:
# graph_path = "../../input/layouts/toScaleLayout.json"
# people_path = "../../input/people/manyPeople.json"
# event_path = "../../input/events/reqReorder.json"
# nurse_log_path = "../../output/nurseLog.csv"
# event_log_path = "../../output/eventLog.csv"

In [3]:
# import src.data_generator as dg
# import src.main
# importlib.reload(src.main)
from src.main import App
from src.data_generator import DataGenerator

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
exp_layout = "../../input/layouts/expLayout.json"
two_nurses = "../../input/people/expPeople1.json" #two nurses, 4 patients each, 2 patients per room
one_nurse = project_root + "/input/people/expOneNurse.json"
exp_nurse_log = project_root + "/output/expNurseLog.csv"
exp_event_log = project_root + "/output/expEventLog.csv"

In [7]:
def generate_exp(exp_event_path, nurse_cnt, request_assigner, include_plans):
    exp_events = project_root + "/input/events/" + exp_event_path
    if nurse_cnt == 1:
        people_file = one_nurse
    elif nurse_cnt == 2:
        people_file = two_nurses
    event_gen = DataGenerator(rnd=42, layout_file=exp_layout, people_file=people_file, out_file=exp_events, request_assigner=request_assigner, include_plans=include_plans)
    event_gen.create_events() 

In [5]:
def run_exp(people_path, event_path):
    app = App(exp_layout, people_path, event_path, exp_nurse_log, exp_event_log)
    app.run_simulation()

In [ ]:
#exp1: two nurses, basic assigner
generate_exp(exp_event_path="expEvents1.json", nurse_cnt=2, request_assigner="basic", include_plans=True)
#exp2: two nurses, other assigner
generate_exp(exp_event_path="expEvents2.json", nurse_cnt=2, request_assigner="other", include_plans=True)
#exp3: one nurse, basic assigner
generate_exp(exp_event_path="expEvents3.json", nurse_cnt=1, request_assigner="basic", include_plans=True)
#exp4: one nurse, other assigner
generate_exp(exp_event_path="expEvents4.json", nurse_cnt=1, request_assigner="other", include_plans=True)
generate_exp(exp_event_path="expEvents5.json", nurse_cnt=2, request_assigner="basic", include_plans=False)
generate_exp(exp_event_path="expEvents6.json", nurse_cnt=2, request_assigner="other", include_plans=False)
generate_exp(exp_event_path="expEvents7.json", nurse_cnt=1, request_assigner="basic", include_plans=False)
generate_exp(exp_event_path="expEvents8.json", nurse_cnt=1, request_assigner="other", include_plans=False)

{'time': 2867.554752696838, 'patient': 0, 'level': 3, 'duration': 139.79909127171078}
{'time': 2155.170543109332, 'patient': 0, 'level': 3, 'duration': 96.87491292803867}
{'time': 359.9096969448104, 'patient': 0, 'level': 3, 'duration': 159.92642186624028}
{'time': 2164.0140422755517, 'patient': 0, 'level': 3, 'duration': 33.08767414437037}
{'time': 3491.6754677831796, 'patient': 0, 'level': 2, 'duration': 61.85086660174142}
{'time': 654.5698819455622, 'patient': 0, 'level': 1, 'duration': 122.62222644415748}
{'time': 2201.9513777578113, 'patient': 0, 'level': 1, 'duration': 94.79175279631737}
{'time': 1048.4249047129508, 'patient': 0, 'level': 3, 'duration': 89.97914575728832}
{'time': 167.99638756901552, 'patient': 0, 'level': 3, 'duration': 98.41049763255539}
{'time': 2226.189633599114, 'patient': 1, 'level': 3, 'duration': 177.48463287101822}
{'time': 1680.3464156927278, 'patient': 1, 'level': 1, 'duration': 121.13172778521576}
{'time': 613.8868452742495, 'patient': 1, 'level': 3, 

In [ ]:
run_exp(people_path=two_nurses, event_path=exp_events1)